# Financial function and classes

## import

In [1]:
from datetime import datetime, timedelta
from dateutil import relativedelta
import pandas as pd

In [ ]:


def get_period_number(self, date):
    """
    Calculate the period number based on the start date of the loan and the number of compounding periods per year.

    Args:
        date (datetime.datetime): The date for which you want to calculate the period number.

    Returns:
        int: The period number.
    """
    delta = date - self.start_date
    days = delta.days
    compounding_days = 365 / self.m
    period_number = days / compounding_days
    return int(period_number)

# Example usage:
start_date = datetime(2023, 1, 1)
date_to_check = datetime(2023, 4, 15)





## Single Cash flow Class

In [2]:
class SingleCashFlow():
    # this makes no sens
    def __init__(self, amount, equivalent_ineterest_rate_per_pmt_period):
        self.P = amount
        self.p = equivalent_ineterest_rate_per_pmt_period
        
    def fv(self, n):
        return self.P * (1 + self.p) ** n
    def pv(self, n):
        return self.P * (1 + self.p) ** -n

### CPG class

In [3]:
class CPG(SingleCashFlow):
    def __init__(self, start_date, investment_horizon_in_years, amount, equivalent_ineterest_rate_per_pmt_period, ):
        self.start_date=start_date
        self.investment_horizon_in_years=investment_horizon_in_years
        super().__init__(amount, equivalent_ineterest_rate_per_pmt_period)
        

## Anuity cashflow class

In [4]:
class GeneralAnnuity:
    """
    Calculate various financial parameters for a general annuity.

    Args:
        principal (float): The initial principal amount.
        nominal_annual_interest_rate (float): The nominal annual interest rate as a decimal.
        number_of_compounding_periods_per_year (int): The number of compounding periods in a year.
        number_of_pmt_per_year (int, optional): The number of payment periods and interest compounding
        in a year in case the pmt do not match. If not provided, it defaults to the same value as `number_of_compounding_periods_per_year`.

    Attributes:
        P (float): The initial principal amount.
        j (float): The nominal annual interest rate as a decimal.
        m (int): The number of interest compounding periods in a year.
        nppy (int): The number of payment periods in a year.

    Properties:
        c (float): The ratio of compounding periods to payment periods.
        i (float): The nominal interest rate per compounding period.
        p (float): The periodic interest rate conversion factor.
    
    Methods:
        sn(n) (float): Calculate the sum of a geometric series for `n` periods.
        an(n) (float): Calculate the annuity factor for `n` periods.
        pmt(n) (float): Calculate the periodic payment amount for `n` periods.
    """
    
    def __init__(self, principal, nominal_annual_interest_rate, number_of_compounding_periods_per_year, number_of_years, number_of_pmt_per_year=None):
        self.P = principal
        self.j = nominal_annual_interest_rate
        self.m = number_of_compounding_periods_per_year
        self.y = number_of_years
        self.nppy = self.m if number_of_pmt_per_year is None else number_of_pmt_per_year
       
    @property
    def last_period(self):
        return self.nppy * self.y
    @property
    def c(self):
        return self.m / self.nppy
    
    @property
    def i(self):
        return self.j / self.m
    
    @property
    def p(self):
        return (1 + self.i) ** self.c - 1

    def sn(self, n):
        return ((1 + self.p) ** n - 1) / self.p
    
    def an(self,n):
        v = (1 / (1 + self.p))
        return (1 - v ** n) / self.p
    @property
    def pmt(self):
        return self.P / self.an(self.last_period)
    


### Class FixedRateFixedAmorizationPeriodLoan

In [5]:
class FixedRateLoan(GeneralAnnuity):
    def __init__(self, name, start_date, term_in_years, principal, nominal_annual_interest_rate, number_of_compounding_periods_per_year, number_of_years):
        self.name = name
        self.term = term_in_years 
        self.start_date = datetime.strptime(start_date, "%Y-%m-%d")
        super().__init__(principal, nominal_annual_interest_rate, number_of_compounding_periods_per_year, number_of_years, number_of_pmt_per_year=None)
    
    @property
    def end_date(self):
        return self.start_date + timedelta(days=365) * self.term
    
    def get_date_for_period(self, period_number):
        """
        Calculate the date corresponding to the given period number based on the start date of the loan and the number of compounding periods per year.

        Args:
            period_number (int): The period number for which you want to calculate the date.

        Returns:
            datetime.datetime: The date corresponding to the period number.
        """
        compounding_days = 365 / self.m
        days = period_number * compounding_days
        date = self.start_date + timedelta(days=days)
        return date
    
    def n(self, evaluation_date):
        """
        Calculate the period number based on the start date of the loan and the number of compounding periods per year.

        Args:
            date (datetime.datetime): The date for which you want to calculate the period number.

        Returns:
            int: The period number.
        """
        if isinstance(evaluation_date, int):
            return evaluation_date
        else:
            evaluation_date = datetime.strptime(evaluation_date, "%Y-%m-%d") if isinstance(evaluation_date, str) else evaluation_date
            assert isinstance(evaluation_date, datetime), f"The evaluation date must be a string or a datetime obj it is a {type(evaluation_date)}"
            delta = evaluation_date - self.start_date
            days = delta.days
            compounding_days = 365 / self.m
            period_number = days / compounding_days
            return int(round(period_number))
    
    def balance(self, evaluation_date=None):
        if not evaluation_date:
            evaluation_date = self.end_date
        n = self.n(evaluation_date)
        return  SingleCashFlow(self.P, self.p).fv(n) - self.pmt * self.sn(n)
    
    def interest_pmt(self, evaluation_date):
        return self.balance(evaluation_date) * self.p
    
    def principal_paiment(self, evaluation_date):
        return self.pmt - self.interest_pmt(evaluation_date)
    
    def interest_cost_as_of(self, evaluation_date=None):
        if not evaluation_date:
            evaluation_date = self.end_date
        return sum([self.interest_pmt(d) for d in range(self.n(evaluation_date) + 1)])
    
    def repaid_principal_as_of(self, evaluation_date):
        return sum([self.principal_paiment(d) for d in range(self.n(evaluation_date) + 1)])

    def get_amortization_schedule(self):
        date_list = []
        balance_list = []
        interest_list = []
        principal_list = []
        for period in range(self.last_period):
            dt = self.get_date_for_period(period).date()
            dt_str = datetime.strftime(dt, "%Y-%m-%d")
            date_list.append(dt)
            balance_list.append(self.balance(dt_str))
            interest_list.append(self.interest_pmt(dt_str))
            principal_list.append(self.principal_paiment(dt_str))
        return pd.DataFrame({"date": date_list, "balance": balance_list, "interest": interest_list, "principal": principal_list})
        
    def modified_duration_remaining_cash_flows(self, new_interest_rate):
        # Calculate the present value of remaining cash flows at the current interest rate
        pv_current_rate = self.balance(self.end_date)

        # Slightly increase the interest rate (e.g., by 1%)
        delta_rate = 0.01
        new_rate = self.p + delta_rate

        # Calculate the present value of remaining cash flows at the new interest rate
        pv_new_rate = 0.01

        # Calculate the change in present values
        delta_pv = pv_new_rate - pv_current_rate

        # Calculate the modified duration
        modified_duration = delta_pv / (2 * pv_current_rate * delta_rate)

        return modified_duration
    
    




#### FixedRateLoan case use

In [10]:
mortgage = FixedRateLoan("House Mortgage", "2023-06-18", 5, 250000, 0.06, 26, 25)
evaluation_date="2023-07-18"
n=mortgage.n(evaluation_date)
mortgage.pmt
print(f"This fixed rate loan contract for {mortgage.P} $ started on the {mortgage.start_date.date()} and will end on the {mortgage.end_date.date()}\
\nfor for a term of {mortgage.term} years. The amortization period was {mortgage.y}.\
\nThe pmt amount was {round(mortgage.pmt)}\
\nThe total amount of interest paid will be {round(mortgage.interest_cost_as_of(mortgage.end_date))} $\
\nand the total principal repayment will be of {round(mortgage.P - mortgage.balance(mortgage.end_date))} $ for an outstanding balance of {round(mortgage.balance(mortgage.end_date))} $\
\nThe Balance is at {round(mortgage.balance(evaluation_date))} as of the {evaluation_date}\
 the amount of interest paid was {round(mortgage.interest_cost_as_of(evaluation_date))} and the\
\nprincipal repaid was {round(mortgage.repaid_principal_as_of(evaluation_date))}. Total interest cost")

# print(f"Value of the loan :{round(SingleCashFlow(mortgage.P, mortgage.p).fv(n))} at periond {n}\
# \nvalue of pmts {round(mortgage.pmt)} value of an {mortgage.sn(n)}")

This fixed rate loan contract for 250000 $ started on the 2023-06-18 and will end on the 2028-06-16
for for a term of 5 years. The amortization period was 25.
The pmt amount was 743
The total amount of interest paid will be 71906 $
and the total principal repayment will be of 25144 $ for an outstanding balance of 224856 $
The Balance is at 249667 as of the 2023-07-18 the amount of interest paid was 1672 and the
principal repaid was 557. Total interest cost


In [ ]:
total_amount = 1100000
amount_1 = 280000
rate_1 = 0.027
amount_2 = total_amount - amount_1
rate_2 = 0.051

loan1 = FixedRateLoan("pret @ 2.7", "2024-04-01", 5, amount_1, rate_1, 26, 30)
loan2 = FixedRateLoan("pret @ 5.1", "2024-04-01", 5, amount_2, rate_2, 26, 30)
loan3 = FixedRateLoan("pret @ 5.1", "2024-04-01", 5, total_amount, rate_2, 26, 30)
t = loan3.interest_cost_as_of() - (loan1.interest_cost_as_of() + loan2.interest_cost_as_of())
amort1 = loan1.get_amortization_schedule()
amort2 = loan2.get_amortization_schedule()
amort3 = loan3.get_amortization_schedule()
t

In [ ]:
ti_1 = amort1["interest"][:5*26].sum()
ti_2 = amort2["interest"][:5*26].sum()
ti_3 = amort3["interest"][:5*26].sum()

In [ ]:
# Specify the Excel file path
excel_file_path = 'data/MC/test2.xlsx'

# Export the DataFrame to Excel
amort3.to_excel(excel_file_path, index=False)

In [ ]:
class RecuringExpenses():
    def __init__(self, name, amount, start_date, pmt_per_year, end_date=None):
        self.name = name
        self.amount = amount
        self.start_date = datetime.strptime(start_date, "%Y-%m-%d")
        self.pmt_per_year = pmt_per_year
        self.end_date = datetime.strptime(end_date, "%Y-%m-%d") if end_date else datetime.today() + timedelta(days=365) * 100
        
    @property
    def annual_amount(self):
        return self.amount * self.pmt_per_year
    @property
    def bi_weekly_amount(self):
        return self.annual_amount / 26
    
    
        
Netflix = RecuringExpenses("Netfilx", 15.75, "2023-02-01", 12)
Netflix.bi_weekly_amount

# Old classes

## Asset Class

In [ ]:
class Asset:
    def __init__(self, name, acquisition_date, acquisition_cost, evaluation_date):
        self.name = name
        self.acquisition_date = datetime.strptime(acquisition_date, "%Y-%m-%d")
        self.acquisition_cost = acquisition_cost
        self.evaluation_date = datetime.strptime(acquisition_date, "%Y-%m-%d") if acquisition_date else datetime.today()
        self.liabilities = []
        self.funding = []
        
    def get_evaluation_date(self, evaluation_date=None):
        if isinstance(evaluation_date, str) or evaluation_date is None:
            return datetime.strptime(evaluation_date, "%Y-%m-%d") if evaluation_date else datetime.now()
        else:
            return evaluation_date

    def add_liability(self, liability):
        assert isinstance(liability, Liability), "Not a liability"
        self.liabilities.append(liability)
        
    def add_funding(self, investment):
        assert isinstance(investment, InvestmentAsset)
        self.funding.append(investment)
        
    def calculate_net_asset_value(self, evaluation_date):
        pass

    def calculate_depreciation(self, evaluation_date=None):
        pass
    
    def calculate_total_costs(self, evaluation_date):
        total_interest_cost = 0
        total_reimbursement_amount = 0
        
        for liability in self.liabilities:
            total_interest_cost += liability.get_info_dict(evaluation_date)["cumulative_interest_costs"]
            total_reimbursement_amount += liability.get_info_dict(evaluation_date)["cumulative_acquisition_cost_repaiments"]
        return total_interest_cost, total_reimbursement_amount

    
    def __str__(self):
        asset_info = f"{self.name}: ${self.acquisition_cost:.2f}, Acquisition Date: {self.acquisition_date.strftime('%Y-%m-%d')}"
        if self.liabilities:
            liability_info = "\n  Liabilities:"
            for liability in self.liabilities:
                liability_info += f"\n    {liability}"
            asset_info += liability_info
        if self.funding:
            funding_info = "\n Funding:"
            for fund in self.funding:
                funding_info += f"\n   {fund}"
            asset_info += funding_info
            
        return asset_info


## Current assets

In [ ]:
class CurrentAsset(Asset):
    def __init__(self, name, acquisition_cost, acquisition_date, residual_value=0):
        super().__init__(name, acquisition_cost, acquisition_date=None)
        self.asset_type = "Current Asset"

    def __str__(self):
        return f"Current {super().__str__()}"

## Fixed Asset

In [ ]:
class FixedAsset(Asset):
    def __init__(self, name, acquisition_cost, useful_life_in_years, acquisition_date=None, residual_value=0):
        super().__init__(name, acquisition_cost, acquisition_date)
        self.asset_type = "Fixed Asset"
        self.useful_life_in_years = useful_life_in_years
        self.residual_value = residual_value

    def set_funding(self, investment):
        self.funding = investment

    def calculate_biweekly_funding(self):
        if self.useful_life_in_years <= 0:
            return 0  # Useful life is already expired
        else:
            total_funding_required = (self.acquisition_cost - self.residual_value) / self.useful_life_in_years
            biweekly_funding = total_funding_required / (self.useful_life_in_years * 26)  # Assuming 26 bi-weekly periods in a year
            return biweekly_funding
        
    def calculate_book_value(self, evaluation_date):
        return self.acquisition_cost - self.calculate_depreciation(evaluation_date)
    
    def calculate_net_asset_value(self, evaluation_date):
        total_liabilities = sum(liability.get_info_dict(evaluation_date)["balance"] for liability in self.liabilities)
        net_asset_value = self.calculate_book_value(evaluation_date) - total_liabilities
        return net_asset_value
    @property
    def annual_depreciation(self):
        return (self.acquisition_cost - self.residual_value) / self.useful_life_in_years
    
    def calculate_depreciation(self, evaluation_date=None):
        age_in_years = (self.get_evaluation_date(evaluation_date) - self.acquisition_date).days / 365
        if age_in_years <= self.useful_life_in_years:
            depreciation = age_in_years * self.annual_depreciation
        else:
            depreciation = self.acquisition_cost
        return depreciation
    
    def calculate_funding(self, evaluation_date):
        pass

    def __str__(self):
        asset_info = f"Fixed {self.name}: ${self.acquisition_cost:.2f}, Useful Life: {self.useful_life_in_years} years, necessary bi-wwekly funding {round(self.calculate_biweekly_funding(), 2)}"
        if self.funding:
            funding_info = f"\n  Funding: {self.funding}"
            return asset_info + funding_info
        return asset_info

## Liability class

In [ ]:
class Liability:
    def __init__(self, name, amount, start_date, nominal_interet_rate, number_of_compounding_periods_per_year, amortisation_period_in_years):
        self.name = name
        self.amount = amount
        self.start_date = datetime.strptime(start_date, "%Y-%m-%d")
        self.nominal_interet_rate = nominal_interet_rate
        self.number_of_compounding_periods_per_year = number_of_compounding_periods_per_year
        self.effective_interest_rate = self.nominal_interet_rate / self.number_of_compounding_periods_per_year
        self.amortisation_period_in_years = amortisation_period_in_years
        self.total_number_of_period = self.amortisation_period_in_years * self.number_of_compounding_periods_per_year
        self.actualisation_factor = (1 - (1 / (1 + self.effective_interest_rate )) ** self.total_number_of_period) / self.effective_interest_rate
        self.pmt = self.amount / self.actualisation_factor
        # self.evaluation_date = datetime.strptime(evaluation_date, "%Y-%m-%d").date() if evaluation_date else None
        
    def generate_amortization_schedule(self):
        amortization_schedule = []
        current_date = self.start_date
        days_per_period = 365.0 / self.number_of_compounding_periods_per_year

        for _ in range(self.total_number_of_period + 1):
            amortization_schedule.append(current_date)
            current_date += timedelta(days=days_per_period)
        # first_date = amortization_schedule.pop(0) 
        return amortization_schedule
    
    def get_closest_previous_date_to_evaluation_date(self, evaluation_date):
        if isinstance(evaluation_date, str):
            evaluation_date = datetime.strptime(evaluation_date, "%Y-%m-%d") if evaluation_date else datetime.now()
        
        end_date = self.start_date + timedelta(days=365) * self.amortisation_period_in_years
        
        assert evaluation_date >= self.start_date, f"Evaluation date can't be inferior to start date received {evaluation_date.date()} while start date is {self.start_date}"
        assert evaluation_date <= end_date, f"Evaluation date must be inferior to end date: {end_date}"
        
        if evaluation_date < self.start_date or evaluation_date > end_date:
            return None
        else:
            date_list = self.generate_amortization_schedule()
            return [date for date in date_list if date_list[date_list.index(date)] <= evaluation_date][-1]
        
    def get_n(self, evaluation_date):
        if isinstance(evaluation_date, str):
            evaluation_date = datetime.strptime(evaluation_date, "%Y-%m-%d") if evaluation_date else datetime.now()
        if evaluation_date and self.start_date:
            days_between_dates = (evaluation_date - self.start_date).days
            periods_between_dates = days_between_dates / (365.0 / self.number_of_compounding_periods_per_year)
            return int(round(periods_between_dates))
        else:
            return None
        
    def sn(self, n):
        return (((1 + self.effective_interest_rate) ** n) - 1) / self.effective_interest_rate
    
    def get_loan_balance(self, evaluation_date):
        n = self.get_n(evaluation_date) 
        return self.amount * ( 1 + self.effective_interest_rate) ** n - self.pmt * self.sn(n)
        
    def get_interest_pmt(self, evaluation_date):
        return self.get_loan_balance(evaluation_date) * self.effective_interest_rate
    
    def get_acquisition_cost_pmt(self, evaluation_date):
        return self.pmt - self.get_interest_pmt(evaluation_date)   
    
    def get_cumulative_interest_costs(self, evaluation_date):
        if isinstance(evaluation_date, str):
            evaluation_date = datetime.strptime(evaluation_date, "%Y-%m-%d") if evaluation_date else datetime.now()

        return sum([self.get_interest_pmt(d) for d in self.generate_amortization_schedule() if d <= evaluation_date])
    
    def get_remaining_interest_costs(self, evaluation_date):
        if isinstance(evaluation_date, str):
            evaluation_date = datetime.strptime(evaluation_date, "%Y-%m-%d") if evaluation_date else datetime.now()
        return sum([self.get_interest_pmt(d) for d in self.generate_amortization_schedule() if d > evaluation_date])
    
    def get_cumulative_acquisition_cost_costs(self, evaluation_date):
        if isinstance(evaluation_date, str):
            evaluation_date = datetime.strptime(evaluation_date, "%Y-%m-%d") if evaluation_date else datetime.now()

        return sum([self.get_acquisition_cost_pmt(d) for d in self.generate_amortization_schedule() if d <= evaluation_date])
    
    def get_info_dict(self, evaluation_date):
        d = self.get_closest_previous_date_to_evaluation_date(evaluation_date)
        return {"date": d.date(), 
                "period": self.get_n(d), 
                "balance": round(self.get_loan_balance(d), 2), 
                "principal_repaiement": round(self.get_acquisition_cost_pmt(d), 2),  
                "interest_cost": round(self.get_interest_pmt(d), 2),  
                "remaining_interets_costs": round(self.get_remaining_interest_costs(d), 2),
                "cumulative_interest_costs": round(car_loan.get_cumulative_interest_costs(d), 2),
                "cumulative_principal_repaiments": round(self.get_cumulative_acquisition_cost_costs(d), 2)
               }
        
        
    def __str__(self):
        return f"{self.name}: ${self.amount:.2f}, monthly pmt:{self.pmt}"


## Investment Class

In [ ]:
class InvestmentAsset(Asset):
    def __init__(self, 
                 name, 
                 acquisition_date, 
                 investment_type, 
                 nominal_interet_rate, 
                 investment_priode_in_years, 
                 acquisition_cost = None,
                 compounding_periods_per_year=1):
        super().__init__(name, acquisition_date, acquisition_cost)
        
        self.asset_type = "Investment"
        self.nominal_interet_rate = nominal_interet_rate
        self.compounding_periods_per_year = compounding_periods_per_year
        self.effective_interest_rate = self.nominal_interet_rate / self.compounding_periods_per_year
        self.investment_priode_in_years = investment_priode_in_years
        self.total_number_of_compunding_periods = self.compounding_periods_per_year * self.investment_priode_in_years
        self.investment_type = investment_type
        self.pmt_date = self.acquisition_date + timedelta(days=365) * investment_priode_in_years
        
    def get_interest_pmt(self):
        return self.acquisition_cost * ((1+self.nominal_interet_rate) ** self.total_number_of_compunding_periods - 1)
        
    def __str__(self):
        return f"Investment {super().__str()}, Type: {self.investment_type}"
    


## Test case

### Car loan

In [ ]:
car_loan = Liability("car loan", 55000, "2020-08-01", 0.06, 12, 5)

In [ ]:
car_loan.get_info_dict("2022-05-25")

#### amortisation schedule

In [ ]:
for d in car_loan.generate_amortization_schedule():
    for k,v in car_loan.get_info_dict(d).items():
        pass

In [ ]:
evaluation_date = "2025-08-01"
# p, i = car_loan.get_cumulative_costs("2025-08-01")
print(car_loan.get_n("2025-08-01"))
print("balance:", round(car_loan.get_loan_balance(evaluation_date)), 
      "pmt:", round(car_loan.pmt), 
      "capital:", round(car_loan.get_acquisition_cost_pmt(evaluation_date)), 
      "interest:", round(car_loan.get_interest_pmt(evaluation_date)))

print("total interest paid", 
      car_loan.get_cumulative_interest_costs(evaluation_date), 
      car_loan.get_cumulative_acquisition_cost_costs(evaluation_date))
# print(p, i)

### Car asset 

#### Instantiation

In [ ]:
car = FixedAsset('Car', 60000, 7, "2020-08-01", residual_value=0)
car_loan = Liability("Car loan", 55000, "2020-08-01", 0.06, 12, 5)
car.add_liability(car_loan)
print(car)

#### Net asset value

In [ ]:
evaluation_date = "2020-10-31"
(car.calculate_net_asset_value(evaluation_date), car.calculate_book_value(evaluation_date))

### Cash flow associated with asset

#### Car depreciation

In [ ]:
car.annual_depreciation / 26

#### Car value at replacement time

In [ ]:
pmt_date = car.acquisition_date + timedelta(days=car.useful_life_in_years * 365)
car_fv = CashFlow(pmt_date, car.acquisition_cost,annual_nominal_interets_rate=0.02, number_of_compounding_period_per_year=1)
car_fv.fv()

#### loan amortisation

In [ ]:
car.acquisition_date + timedelta(days=car.useful_life_in_years * 365)

for d in car.liabilities[0].generate_amortization_schedule():
    print(car.liabilities[0].get_info_dict(d)["date"], car.liabilities[0].get_info_dict(d)['principal_repaiement'] + car.liabilities[0].get_info_dict(d)['interest_cost'] )
# car.calculate_depreciation()
# car.useful_life_in_years

### Funding


In [ ]:
cpg = InvestmentAsset("Car Funding", 5000, "CPG", 0.06, 3, acquisition_cost="2022-07-29")
car.add_funding(cpg)

In [ ]:
print(round(cpg.get_interest_pmt(), 2), cpg.pmt_date.date())

In [ ]:
car_purchased_date = car.acquisition_date + timedelta(days=365) * car.useful_life_in_years
round(car.calculate_depreciation())
# car_purchased_date

In [ ]:
maison = FixedAsset("709 rue Beaudoin", 350000, 120, "2018-02-01")

In [ ]:
revetement_bardeau = FixedAsset("Revêtement Bardeau", 15000, 15, "")

# Cash flow class

In [ ]:
 amount = car.acquisition_cost

# Open ended annuity

The open ended annuity is used to model recurring spending item at regular time interval where 
the start date 
the pmt 
the frequency
the end date (if unknown the difference between the age and expected age at death is used)

the user info is needed for this.

the interest rate

In [ ]:
class OpenEndedAnnuity(GeneralAnnuity):
    def __init__(self):
        super.__init__(nominal_annual_interest_rate, number_of_compounding_periods_per_year, number_of_years, number_of_pmt_per_year=None, principal=None)
    

In [ ]:
appleCloud = OpenEndedAnnuity()